<h1 style="color:black;">SuperCarros Car Scrapper</h1>

<t1 style="color:black;"> This is a web scraper designed to extract detailed information about cars listed on Supercarros.com, a popular online marketplace in the Dominican Republic. The scraper collects key data such as car make, model, year, price, mileage, and other relevant details to facilitate analysis and comparison.</t1>

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

base_url = "https://www.supercarros.com/buscar/"
base_params = "?do=1&ObjectType=1&PriceFrom=0&PriceTo=50000000&YearFrom=1970&YearTo=2025&Brand="

driver.get("https://www.supercarros.com/carros/cualquier-tipo/cualquier-provincia/ver-todos/")
time.sleep(3)

In [3]:
try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "ul.bigsearch-filters-box-list li"))
    )
    print("Brand list loaded.")
except Exception as e:
    print(f"Error loading brand list: {e}")
    driver.quit()
    exit()

Brand list loaded.


In [4]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul.bigsearch-filters-box-list li"))
)

# Extract car brand names and ID.
brand_elements = driver.find_elements(By.CSS_SELECTOR, "ul.bigsearch-filters-box-list li")
brands = []
for brand in brand_elements:
    data_label = brand.get_attribute("data-label")
    data_id = brand.get_attribute("data-id")
    if data_label and data_id:
        brands.append({
            "name": data_label.strip(),
            "id": data_id.strip()
        })

print(f"Found {len(brands)} brands.")


Found 110 brands.


In [5]:
# List to store all scraped car data
all_car_data = []
max_cars_per_brand = 1000 

for i, brand in enumerate(brands):
    brand_name = brand["name"]
    brand_id = brand["id"]
    print(f"Processing brand {i + 1}/{len(brands)}: {brand_name} (ID: {brand_id})")

    # Build URL + Parameters car brands
    brand_url = f"{base_url}{base_params}{brand_id}"
    driver.get(brand_url)
    time.sleep(3)

    brand_car_data = []
    current_page = 0

    while len(brand_car_data) < max_cars_per_brand:
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li[class*='normal']"))
            )
        except Exception as e:
            print(f"No cars found on page {current_page + 1} for {brand_name}: {e}")
            break

        # Locate car containers on the current page
        car_containers = driver.find_elements(By.CSS_SELECTOR, "li[class*='normal']")
        print(f"Found {len(car_containers)} cars on page {current_page + 1} for {brand_name}.")

        for car in car_containers:
            try:
                
                price = car.find_element(By.CLASS_NAME, "price").text
                year = car.find_element(By.CLASS_NAME, "year").text
                title1 = car.find_element(By.CLASS_NAME, "title1").text  # Vehicle name
                title2 = car.find_element(By.CLASS_NAME, "title2").text  # Additional info (fuel, condition)
                image_url = car.find_element(By.CLASS_NAME, "real").get_attribute("src")
        
                brand_car_data.append({
                    "Brand": brand_name,
                    "Price": price,
                    "Year": year,
                    "Vehicle Name": title1,
                    "Details": title2,
                    "Image URL": image_url
                })

                # Stop when reach the limit
                if len(brand_car_data) >= max_cars_per_brand:
                    break
            except Exception as e:
                print(f"Error extracting data for a car: {e}")
                continue

        current_page += 1
        try:
            next_page_button = driver.find_element(By.LINK_TEXT, str(current_page + 1))
            next_page_button.click()
            time.sleep(3)
        except Exception as e:
            print(f"No more pages for {brand_name}.")
            break

    all_car_data.extend(brand_car_data)
    print(f"Collected {len(brand_car_data)} cars for {brand_name}.")


Processing brand 1/111: Acura (ID: 36)
Found 30 cars on page 1 for Acura.
Error extracting data for a car: Message: no such element: Unable to locate element: {"method":"css selector","selector":".price"}
  (Session info: chrome=131.0.6778.265); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0037FD53+23747]
	(No symbol) [0x00307D54]
	(No symbol) [0x001DBE53]
	(No symbol) [0x0021FCA6]
	(No symbol) [0x0021FEEB]
	(No symbol) [0x00215A81]
	(No symbol) [0x00241E44]
	(No symbol) [0x002159A4]
	(No symbol) [0x00242094]
	(No symbol) [0x0025B41E]
	(No symbol) [0x00241B96]
	(No symbol) [0x00213F3C]
	(No symbol) [0x00214EBD]
	GetHandleVerifier [0x0065AC73+3017699]
	GetHandleVerifier [0x0066B93B+3086507]
	GetHandleVerifier [0x006640F2+3055714]
	GetHandleVerifier [0x00415AF0+637536]
	(No symbol) [0x00310A5D]
	(No symbol) [0x0030DA28]
	(No symbol) [0x0030DBC5]
	(No symbo

No more pages for Acura.
Collected 35 cars for Acura.
Processing brand 2/111: Aiqar (ID: 791)
Found 10 cars on page 1 for Aiqar.
Error extracting data for a car: Message: no such element: Unable to locate element: {"method":"css selector","selector":".price"}
  (Session info: chrome=131.0.6778.265); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0037FD53+23747]
	(No symbol) [0x00307D54]
	(No symbol) [0x001DBE53]
	(No symbol) [0x0021FCA6]
	(No symbol) [0x0021FEEB]
	(No symbol) [0x00215A81]
	(No symbol) [0x00241E44]
	(No symbol) [0x002159A4]
	(No symbol) [0x00242094]
	(No symbol) [0x0025B41E]
	(No symbol) [0x00241B96]
	(No symbol) [0x00213F3C]
	(No symbol) [0x00214EBD]
	GetHandleVerifier [0x0065AC73+3017699]
	GetHandleVerifier [0x0066B93B+3086507]
	GetHandleVerifier [0x006640F2+3055714]
	GetHandleVerifier [0x00415AF0+637536]
	(No symbol) [0x00310A5D]
	(No sy

Found 18 cars on page 1 for Alfa Romeo.
Error extracting data for a car: Message: no such element: Unable to locate element: {"method":"css selector","selector":".price"}
  (Session info: chrome=131.0.6778.265); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0037FD53+23747]
	(No symbol) [0x00307D54]
	(No symbol) [0x001DBE53]
	(No symbol) [0x0021FCA6]
	(No symbol) [0x0021FEEB]
	(No symbol) [0x00215A81]
	(No symbol) [0x00241E44]
	(No symbol) [0x002159A4]
	(No symbol) [0x00242094]
	(No symbol) [0x0025B41E]
	(No symbol) [0x00241B96]
	(No symbol) [0x00213F3C]
	(No symbol) [0x00214EBD]
	GetHandleVerifier [0x0065AC73+3017699]
	GetHandleVerifier [0x0066B93B+3086507]
	GetHandleVerifier [0x006640F2+3055714]
	GetHandleVerifier [0x00415AF0+637536]
	(No symbol) [0x00310A5D]
	(No symbol) [0x0030DA28]
	(No symbol) [0x0030DBC5]
	(No symbol) [0x003007F0]
	BaseThreadInitThu

KeyboardInterrupt: 

In [6]:
df = pd.DataFrame(all_car_data)
df.to_csv("car_data_by_brand.csv", index=False)

driver.quit()

print(f"Scraping completed. Collected data for all brands. Total cars scraped: {len(all_car_data)}.")

Scraping completed. Collected data for all brands. Total cars scraped: 17580.
